# Automatic Short Answer Scoring System

### This system can automatically score Student Text Answers by supplying the system with reference answer-student answer pairs in a csv file. 

###### Note: Please run each cell below - one by one and one after the other. Enter in the details after running the each respective cell.

In [ ]:
#@title
%%capture
!pip install sentence-transformers
!pip install transformers
!pip install ipyfilechooser
!pip install pivottablejs
from pivottablejs import pivot_ui
from IPython.display import HTML
import pandas as pd
import numpy as np
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel, pipeline, AutoConfig, AdamW, RobertaModel, RobertaConfig, RobertaTokenizer, AutoModelForSequenceClassification
from scipy import spatial
import torch
from sentence_transformers import SentenceTransformer, util
from keras.preprocessing.sequence import pad_sequences
from torch import nn
import time
import random
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import get_linear_schedule_with_warmup
import datetime
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import os
import regex as re
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets
from ipyfilechooser import FileChooser
import base64
from IPython.display import HTML
import io
import math

In [ ]:
#@title
# Source: https://github.com/groundhogday321/python-ipywidgets/blob/master/Python%20Add%20Interactivity%20to%20Jupyter%20Notebooks%20with%20ipywidgets.ipynb
text = ipywidgets.Text(value='',
                       placeholder='Type Dataset Name Here',
                       description='Text:',
                       disabled=False)

instructions = ipywidgets.widgets.HTML('Enter Dataset Name:')
display(instructions)

display(text)

def submit(dataset):
    print("Thank You. Please Note: This name would be used for names of files in storage")

text.on_submit(submit)

HTML(value='Enter Dataset Name:')

Text(value='', description='Text:', placeholder='Type Dataset Name Here')

Thank You. Please Note: This name would be used for names of files in storage


In [ ]:
#@title
dataset_name = text.value
# Source: https://www.codegrepper.com/code-examples/python/download+csv+file+from+jupyter+notebook
def create_download_link(df, title = "ASAG CSV Format", filename = "ASAG_CSV_Format.csv"):
    csv = df.to_csv(index = False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)
instructions = ipywidgets.widgets.HTML('Would you like to train the system on a new dataset?')
display(instructions)

# generate password

drop_down = ipywidgets.Dropdown(options=['','Yes', 'No'],
                                description='Answer:',
                                disabled=False)
def Get_Requirement(option): 
  if option == '':
    return None
  elif option == "Yes":
    instructions = ipywidgets.widgets.HTML('Please find attached below the CSV format for training the system:')
    display(instructions)
    train_data = {'Student_Answers':["Student Answer 1", "Student Answer 2", "Student Answer 3", "..."],
            'Reference_Answers':["Reference Answer 1", "Reference Answer 2", "Reference Answer 3", "..."],
            'Scores':['Score 1', "Score 2", "Score 3", "..."]}
    # Create DataFrame
    train_format = pd.DataFrame(train_data)
    # Print the output.
    train_format.to_csv("ASAG_Train_Format.csv", index = False)
    display(create_download_link(train_format, "ASAG Train Format", "ASAG_Train_Format.csv"))
  else:
    print("Thank you. Please move on to the next cell.")
try:
  ipywidgets.interact(Get_Requirement, option=drop_down);
except Exception as e:
  print("Please make sure you have run all cells above before running this cell")
  print(e)

HTML(value='Would you like to train the system on a new dataset?')

interactive(children=(Dropdown(description='Answer:', options=('', 'Yes', 'No'), value=''), Output()), _dom_cl…

In [ ]:
#@title
try:
  Training_First_Time = drop_down.value
  instructions = ipywidgets.widgets.HTML('Please find attached the CSV format for scoring the Answers:')
  display(instructions)
  test_data = {'Student_Answers':["Student Answer 1", "Student Answer 2", "Student Answer 3", "..."],
            'Reference_Answers':["Reference Answer 1", "Reference Answer 2", "Reference Answer 3", "..."]}
  # Create DataFrame
  test_format = pd.DataFrame(test_data)
  test_format.to_csv("ASAG_Scoring_Format.csv", index = False)
  display(create_download_link(test_format, "ASAG Scoring File Format", 'ASAG_Scoring_Format.csv'))
except Exception as e:
  print("Please make sure you have run all the cells above before running this cell")
  print(e)

HTML(value='Please find attached the CSV format for scoring the Answers:')

In [ ]:
#@title
try:
  if Training_First_Time == "Yes":
    instructions = ipywidgets.widgets.HTML('Please upload CSV file for the system to learn from:')
    display(instructions)
    uploader = ipywidgets.FileUpload(accept='*.csv', multiple=False)
    display(uploader)
  else: 
    print("Thank you for running this cell. Please move on to the next cell")
except Exception as e:
  print("Please make sure you have run all the cells above before running this cell")
  print(e)

HTML(value='Please upload CSV file for the system to learn from:')

FileUpload(value={}, accept='*.csv', description='Upload')

In [ ]:
#@title
try: 
  # Source: https://www.engineerknow.com/2021/06/how-to-upload-csv-file-in-jupyter.html
  if Training_First_Time == "Yes":
    input_file = list(uploader.value.values())[0]
    content = input_file['content']
    content = io.StringIO(content.decode('utf-8'))
    df_train = pd.read_csv(content)
  instructions = ipywidgets.widgets.HTML('Please upload CSV file for scoring the answers:')
  display(instructions)
  uploader = ipywidgets.FileUpload(accept='*.csv', multiple=False)
  display(uploader)
except Exception as e:
  print("Please make sure you have run all the cells above before running this cell")
  print(e)

HTML(value='Please upload CSV file for scoring the answers:')

FileUpload(value={}, accept='*.csv', description='Upload')

In [ ]:
#@title
try:
  input_file = list(uploader.value.values())[0]
  content = input_file['content']
  content = io.StringIO(content.decode('utf-8'))
  df_test = pd.read_csv(content)
  OG_test = df_test.copy()
  instructions = ipywidgets.widgets.HTML('Please select folder of file storage:')
  display(instructions)
  fc = FileChooser()
  display(fc)
except Exception as e:
  print("Please make sure you have run all the cells above before running this cell")
  print(e)

HTML(value='Please select folder of file storage:')

FileChooser(path='/content', filename='', title='HTML(value='', layout=Layout(display='none'))', show_hidden='…

In [ ]:
#@title
folder_path = fc.selected_path
if Training_First_Time == "Yes":
  instructions = ipywidgets.widgets.HTML('This may take a couple of hours since you are training on this dataset for first time. Please be patient')
  display(instructions)
else:
  instructions = ipywidgets.widgets.HTML('This may take less than half an hour. Please be patient')
  display(instructions)
  try: 
    df_train = pd.read_csv(f"{folder_path}/{dataset_name}.csv")
  except Exception as e:
    print("Please correct the dataset name or the folder path specified above. Either of the two are incorrect. Once done, please rerun this notebook from start")
    print(e)

HTML(value='This may take a couple of hours since you are training on this dataset for first time. Please be p…

In [ ]:
#@title
from IPython.utils import io # importing this here because io has a name conflict above
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))
def Prepare_Training_Data(sent1, sent2, tokenizer, batch_size, labels = None):
  Lengths = []
  inputs = []

  MAX_LEN = 256
  # For every sentence...
  for i in range(len(sent1)):
      # `encode` will:
      #   (1) Tokenize the sentence.
      #   (2) Prepend the `[CLS]` token to the start.
      #   (3) Append the `[SEP]` token to the end.
      #   (4) Map tokens to their IDs.
      encoded_sent = tokenizer.encode(
                          sent1[i],
                          sent2[i],     
                          truncation = True,                 
                          add_special_tokens = True, 
                          max_length=MAX_LEN
                    )
      inputs.append(encoded_sent)    
      Lengths.append(len(encoded_sent))


  # Set the maximum sequence length.
  # I've chosen 64 somewhat arbitrarily. It's slightly larger than the
  # maximum training sentence length of 47...
  

  # Pad our input tokens with value 0.
  # "post" indicates that we want to pad and truncate at the end of the sequence,
  # as opposed to the beginning.
  inputs = pad_sequences(inputs, maxlen=MAX_LEN, dtype="long", 
                            value=0, truncating="post", padding="post")


  # Create attention masks
  masks = []

  # For each sentence...
  for sent in inputs:
      
      # Create the attention mask.
      #   - If a token ID is 0, then it's padding, set the mask to 0.
      #   - If a token ID is > 0, then it's a real token, set the mask to 1.
      att_mask = [int(token_id > 0) for token_id in sent]
      
      # Store the attention mask for this sentence.
      masks.append(att_mask)

  # Convert all inputs and labels into torch tensors, the required datatype 
  # for our model.
  inputs = torch.tensor(inputs)
  masks = torch.tensor(masks)
  if labels is not None:
    labels = torch.tensor(labels)

  # Create the DataLoader for our training set.
  if labels is not None:
    data = TensorDataset(inputs, masks, labels)
  else:
    data = TensorDataset(inputs, masks)
  sampler = RandomSampler(data)
  dataloader = DataLoader(data, sampler=sampler, batch_size=batch_size)

  return dataloader

def Prepare_Pred_Data(sent1, sent2, tokenizer, batch_size, labels = None):
  MAX_LEN = 256
  # Create sentence and label lists

  # Tokenize all of the sentences and map the tokens to thier word IDs.
  input_ids = []

  # For every sentence...
  for i in range(len(sent1)):
      # `encode` will:
      #   (1) Tokenize the sentence.
      #   (2) Prepend the `[CLS]` token to the start.
      #   (3) Append the `[SEP]` token to the end.
      #   (4) Map tokens to their IDs.
      encoded_sent = tokenizer.encode(
                          sent1[i],
                          sent2[i],
                          truncation = True, 
                          add_special_tokens = True # Add '[CLS]' and '[SEP]'
                    )
      
      input_ids.append(encoded_sent)

  # Pad our input tokens
  input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, 
                            dtype="long", truncating="post", padding="post")

  # Create attention masks
  attention_masks = []

  # Create a mask of 1s for each token followed by 0s for padding
  for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask) 

  # Convert to tensors.
  prediction_inputs = torch.tensor(input_ids)
  prediction_masks = torch.tensor(attention_masks)
  if labels is not None: 
    prediction_labels = torch.tensor(labels)


  # Create the DataLoader.
  if labels is not None: 
    prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
  else: 
    prediction_data = TensorDataset(prediction_inputs, prediction_masks)
  prediction_sampler = SequentialSampler(prediction_data)
  prediction_dataloader = DataLoader(prediction_data,  sampler=prediction_sampler, batch_size=batch_size)

  return prediction_dataloader

class Large_Arch(nn.Module):

    def __init__(self, model):
      
      super(Large_Arch, self).__init__()

      self.model = model
      
      # dropout layer
      self.dropout = nn.Dropout(0.1)
      
      # relu activation function
      self.relu =  nn.ReLU()

      #output
      self.output = nn.Linear(1024,1)

    #define the forward pass
    def forward(self, sent_id, mask):

      #pass the inputs to the model  
      _, cls_hs= self.model(sent_id, attention_mask=mask, return_dict=False)
      
      x = self.dropout(cls_hs)

      x = self.relu(x)

      x = self.output(x)

      return x
# model_name = 'huggingface/CodeBERTa-small-v1'
# train_sent1 = df_train['Student_Answers'].values
# train_sent2 = df_train['Reference_Answers'].values
# train_labels = df_train['Scores'].values
# test_sent1=df_test['Student_Answers'].values
# test_sent2 = df_test['Reference_Answers'].values
# test_labels = df_test['Scores'].values
# val_sent1 = df_val['Student_Answers'].values
# val_sent2 = df_val['Reference_Answers'].values
# val_labels = df_val['Scores'].values
# lr = 1e-5
# epochs = 2

def Fine_Tune(model_name, train_sent1, train_sent2, train_labels, test_sent1, test_sent2,val_sent1, val_sent2, val_labels, df_train, df_test, df_val):
  batch_size = 1
  epochs = 10
  lr = 1e-5
  weight_decay = 0.1
  warmup_proportion = 0.06
  # gradient_accumulations = 1
  # scaler = GradScaler()
  with io.capture_output() as captured:
    model = AutoModel.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=True, return_token_type_ids = True)
  # Tokenize all of the sentences and map the tokens to thier word IDs.
  train_dataloader = Prepare_Training_Data(train_sent1, train_sent2, tokenizer, batch_size, train_labels)
  test_dataloader = Prepare_Training_Data(test_sent1, test_sent2, tokenizer, batch_size = batch_size)
  validation_dataloader = Prepare_Training_Data(val_sent1, val_sent2, tokenizer, batch_size, val_labels)


  # freeze all the parameters
  for param in model.parameters():
      param.requires_grad = True

  if ("large" in model_name):  
    # pass the pre-trained BERT to our define architecture
    model = Large_Arch(model)
  else:
    model = Base_Arch(model)

  # push the model to GPU
  model.cuda()

  optimizer = AdamW(model.parameters(), 
                  lr = lr, weight_decay = weight_decay, correct_bias = False
                )

  total_steps = len(train_dataloader)/batch_size * epochs
  num_warmup_steps = total_steps*warmup_proportion

  # Create the learning rate scheduler.
  scheduler = get_linear_schedule_with_warmup(optimizer, 
                                              num_warmup_steps = num_warmup_steps, 
                                              num_training_steps = total_steps)


  # This training code is based on the `run_glue.py` script here:
  # https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128
  # Source: https://www.analyticsvidhya.com/blog/2020/07/transfer-learning-for-nlp-fine-tuning-bert-for-text-classification/


  # Set the seed value all over the place to make this reproducible.
  seed_val = 42

  random.seed(seed_val)
  np.random.seed(seed_val)
  torch.manual_seed(seed_val)
  torch.cuda.manual_seed(seed_val)
  torch.cuda.manual_seed_all(seed_val)
  # torch.backends.cudnn.deterministic = True

  # Store the average loss after each epoch so we can plot them.
  train_loss_values = []
  valid_loss_values = []
  total_preds = []
  total_val_preds = []
  total_train_preds = []

  # set initial loss to infinite
  best_valid_loss = float('inf')

  #Defining the loss function
  loss_fn = nn.HuberLoss()
  mse_loss = nn.MSELoss()

  model.double()

  # For each epoch...
  for epoch_i in tqdm(range(0, epochs)):
      
      # ========================================
      #               Training
      # ========================================
      
      # Perform one full pass over the training set.


      # Measure how long the training epoch takes.
      t0 = time.time()

      # Reset the total loss for this epoch.
      total_loss = 0
      total_loss = np.float128(total_loss)

      # Put the model into training mode. Don't be mislead--the call to 
      # `train` just changes the *mode*, it doesn't *perform* the training.
      # `dropout` and `batchnorm` layers behave differently during training
      # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
      model.train()

      model.zero_grad()
      # For each batch of training data...
      for step, batch in enumerate(train_dataloader):

          # Progress update every 40 batches.
          if step % 40 == 0 and not step == 0:
              # Calculate elapsed time in minutes.
              elapsed = format_time(time.time() - t0)
              
              # Report progress.

          # Unpack this training batch from our dataloader. 
          #
          # As we unpack the batch, we'll also copy each tensor to the GPU using the 
          # `to` method.
          #
          # `batch` contains three pytorch tensors:
          #   [0]: input ids 
          #   [1]: attention masks
          #   [2]: labels 
          b_input_ids = batch[0].to(device)
          b_input_mask = batch[1].to(device)
          b_labels = batch[2].to(device)
          b_labels = b_labels.view(-1,1)
          
          # Always clear any previously calculated gradients before performing a
          # backward pass. PyTorch doesn't do this automatically because 
          # accumulating the gradients is "convenient while training RNNs". 
          # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
          
          
          # with autocast():
          # get model predictions for the current batch
          preds = model(b_input_ids, b_input_mask)
          # compute the loss between actual and predicted values
          loss = loss_fn(preds, b_labels)

          # Accumulate the training loss over all of the batches so that we can
          # calculate the average loss at the end. `loss` is a Tensor containing a
          # single value; the `.item()` function just returns the Python value 
          # from the tensor.
          total_loss += loss.item()
          
          
          # Perform a backward pass to calculate the gradients.
          # scaler.scale(loss / gradient_accumulations).backward()
          loss.backward()
          # Clip the norm of the gradients to 1.0.
          # This is to help prevent the "exploding gradients" problem.
          torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

          
          # if (step + 1) % gradient_accumulations == 0:
          # Update parameters and take a step using the computed gradient.
          # The optimizer dictates the "update rule"--how the parameters are
          # modified based on their gradients, the learning rate, etc.
          # scaler.step(optimizer)
          optimizer.step()
          # scaler.update()
          # Update the learning rate.
          scheduler.step()
          model.zero_grad()

          # model predictions are stored on GPU. So, push it to CPU
          preds=preds.detach().cpu().numpy()
          
          # append the model predictions
          total_preds.append(preds)
        
      # Calculate the average loss over the training data.
      avg_train_loss = total_loss / len(train_dataloader)            
      
      # Store the loss value for plotting the learning curve.
      # train_loss_values.append(avg_train_loss)
      # # predictions are in the form of (no. of batches, size of batch, no. of classes).
      # # reshape the predictions in form of (number of samples, no. of classes)
      # total_preds  = np.concatenate(total_preds, axis=0)
      
      
      del b_input_ids, b_input_mask, b_labels, batch

      # ========================================================
      #               Validation and Calculating Training Loss
      # ========================================================
      # After the completion of each training epoch, measure our performance on
      # our validation set.


      # Reset the total loss for this epoch.
      total_val_loss = 0
      total_val_loss = np.float128(total_val_loss)
      total_val_rmse = 0
      total_val_rmse = np.float128(total_val_rmse)
      total_train_loss = 0
      total_train_loss = np.float128(total_train_loss)
      
      t0 = time.time()

      # Put the model in evaluation mode--the dropout layers behave differently
      # during evaluation.
      model.eval()

      # Tracking variables 
      
      nb_eval_steps = 0
      for val_batch in validation_dataloader:
          
          # Add batch to GPU
          val_batch = tuple(t.to(device) for t in val_batch)
          
          
          # Unpack the inputs from our dataloader
          v_input_ids, v_input_mask, v_labels = val_batch
          
          v_labels = v_labels.view(-1,1)
          
          
          # Telling the model not to compute or store gradients, saving memory and
          # speeding up validation
          with torch.no_grad():        

              # Forward pass, calculate logit predictions.
              # This will return the logits rather than the loss because we have
              # not provided labels.
              # token_type_ids is the same as the "segment ids", which 
              # differentiates sentence 1 and 2 in 2-sentence tasks.
              # The documentation for this `model` function is here: 
              # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
              val_preds = model(v_input_ids, 
                              # token_type_ids=None, 
                              v_input_mask)
          


              # compute the validation loss between actual and predicted values
              eval_loss = loss_fn(val_preds, v_labels)
              rmse = torch.sqrt(mse_loss(val_preds, v_labels))
          

              total_val_loss = total_val_loss + eval_loss.item()
              total_val_rmse = total_val_rmse+rmse.item()
          

              val_preds = val_preds.detach().cpu().numpy()
          

              total_val_preds.append(val_preds)
          

              

          # Track the number of batches
          nb_eval_steps += 1
      
      # # reshape the predictions in form of (number of samples, no. of classes)
      # total_val_preds  = np.concatenate(total_val_preds, axis=0)
      
      
      # compute the validation loss of the epoch
      avg_val_loss = total_val_loss / nb_eval_steps
      avg_val_rmse = total_val_rmse / nb_eval_steps
    
      valid_loss_values.append(avg_val_loss)
      
      if avg_val_loss < best_valid_loss:
        checkpoint = {"epoch":epoch_i, "model_state":model.state_dict(), "optim_state": optimizer.state_dict()}
        if "/" in model_name:
          torch.save(checkpoint, f"{folder_path}/{model_name.split('/')[1]}_{dataset_name}_Checkpoint.pth.tar")
          if (avg_val_rmse >= 0.15) and (epoch_i >= 6):
            break
        else: 
          torch.save(checkpoint, f"{folder_path}/{model_name}_{dataset_name}_Checkpoint.pth.tar")
          if (avg_val_rmse >= 0.15) and (epoch_i >= 6):
            break
        best_valid_loss = avg_val_loss  
  train_dataloader = Prepare_Pred_Data(train_sent1, train_sent2, tokenizer, batch_size, train_labels)
  test_dataloader = Prepare_Pred_Data(test_sent1, test_sent2, tokenizer, batch_size)
  validation_dataloader = Prepare_Pred_Data(val_sent1, val_sent2, tokenizer, batch_size,val_labels)

  return train_dataloader, test_dataloader, validation_dataloader

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def Get_Preds(dataloader, model, Test = False):
  preds = []
  labels = []
  mse = []
  model.eval()
  loss_fn = nn.MSELoss()
  for batch in dataloader:
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    if Test == False: 
      b_labels = batch[2].to(device)
      b_labels = b_labels.view(-1,1)
    with torch.no_grad():
      pred = model(b_input_ids, b_input_mask)
      if Test == False:
        loss = torch.sqrt(loss_fn(pred, b_labels))
      pred = pred.detach().cpu().numpy()
      if Test == False:
        label_ids = b_labels.to('cpu').numpy()
    if Test == False:
      mse.append(loss.item())
    preds.append(pred[0][0])
    if Test == False:
      labels.append(label_ids[0][0])
  return preds, labels, mse

# sent1 = df_test['Student_Answers'].values
# sent2 = df_test['Reference_Answers'].values
# path = f"{folder_path}/{model_name}_{dataset_name}_Checkpoint.pth.tar"
def Preds_From_Checkpoint(model_name, batch_size, path, sent1 = None, sent2 = None, train_dataloader = None, test_dataloader = None, validation_dataloader = None):
  with io.capture_output() as captured:
    model = AutoModel.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=True, return_token_type_ids = True)
  # Tokenize all of the sentences and map the tokens to thier word IDs.
  for param in model.parameters():
      param.requires_grad = False
  model = Large_Arch(model)
  try:
    checkpoint = torch.load(path)
  except Exception as e:
    print("Please correct the dataset name or the folder path specified above. Either of the two are incorrect. Once done, please rerun this notebook from start")
    print(e)
  model.load_state_dict(checkpoint['model_state'], strict = False)

  model.to(device)
  preds = []
  model.eval()
  
  if test_dataloader == None:
    test_dataloader = Prepare_Pred_Data(sent1, sent2, tokenizer, batch_size)
    for batch in tqdm(test_dataloader):
      b_input_ids = batch[0].to(device)
      b_input_mask = batch[1].to(device)
      with torch.no_grad():
        pred = model(b_input_ids, b_input_mask)
        pred = pred.detach().cpu().numpy()
      preds.append(pred[0][0])
    return preds
  else: 
    test_preds, _, _ = Get_Preds(test_dataloader, model, True)
    train_preds,train_labels, train_mse = Get_Preds(train_dataloader, model)
    val_preds,val_labels, val_mse = Get_Preds(validation_dataloader, model)
    return train_preds, val_preds, test_preds  

Max_Score = max(df_train['Scores'].values)
scaler = MinMaxScaler()
df_train['Scores'] = scaler.fit_transform(df_train[["Scores"]])

SA = df_train['Student_Answers'].values
SA_len = []
for i in SA:
  SA_len.append(len(i.split()))
# Finding the longest sentence in the Reference_Answers
RA = df_train['Reference_Answers'].values
RA_len = []
for i in RA:
  RA_len.append(len(i.split()))
if max(SA_len)+max(RA_len)>250:
  print("Warning: The sentences in the answers are too long and do not categorize into short answers. Would suggest to get support on this.")

device = torch.device("cuda")
correlation = 0
m = 111
Random_Number = 40

model_name = 'cross-encoder/stsb-roberta-large'
path = f"{folder_path}/{model_name.split('/')[1]}_{dataset_name}_Checkpoint.pth.tar"
if Training_First_Time== "No":
  df_test[f'Finetuned {model_name} Preds'] = Preds_From_Checkpoint(model_name, 1, path, df_test['Student_Answers'].values, df_test['Reference_Answers'].values)
else:
  while correlation<=0.75:  
    #creating train and test dictionaries stratified for each question
    train_dict = {}
    val_dict = {}
    for key in np.unique(df_train.Reference_Answers):
      Ques_df = df_train.loc[df_train['Reference_Answers']==key]
      val_row = Ques_df.sample(random_state = Random_Number, n = math.ceil(0.10*len(Ques_df)))
      train_rows = pd.merge(Ques_df,val_row, indicator=True, how='outer').query('_merge=="left_only"').drop('_merge', axis=1)
      train_dict[key] = train_rows
      val_dict[key] = val_row
    df_val = pd.DataFrame(columns = np.array(df_train.columns))
    for i in val_dict:
      df_val = pd.concat([df_val, val_dict[i]])
    df_val.head()
    df_train = pd.DataFrame(columns = np.array(df_train.columns))
    for i in train_dict:
      df_train = pd.concat([df_train, train_dict[i]])
    # Finding the longest sentence in the Student_Answers
    train_dataloader, test_dataloader, validation_dataloader = Fine_Tune(model_name, 
                                          df_train['Student_Answers'].values, df_train['Reference_Answers'].values, df_train['Scores'].values,
                                          df_test['Student_Answers'].values, df_test['Reference_Answers'].values, 
                                          df_val['Student_Answers'].values, df_val['Reference_Answers'].values, df_val['Scores'].values, 
                                          df_train, df_test, df_val)  
    train_preds, val_preds, test_preds = Preds_From_Checkpoint(model_name, 1, path, None, None, train_dataloader, test_dataloader, validation_dataloader)
    df_train[f'Finetuned {model_name} Preds'] = train_preds
    df_test['Scores'] = test_preds
    df_val[f'Finetuned {model_name} Preds'] = val_preds
    correlation = df_val['Scores'].corr(df_val[f'Finetuned {model_name} Preds'])
    df_train.to_csv(f"{folder_path}/{model_name.split('/')[1]}_df_train_{dataset_name}.csv", index = False)
    df_val.to_csv(f"{folder_path}/{model_name.split('/')[1]}_df_val_{dataset_name}.csv", index = False)
    Random_Number = Random_Number+m
df_train = pd.concat([df_train,df_val])
df_test.Scores = df_test.Scores*Max_Score
df_test['Scores'] = df_test['Scores'].where(df_test['Scores'] <= Max_Score, Max_Score)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
left_aligned_df_test = df_test.style.set_properties(**{'text-align': 'left'})
display(left_aligned_df_test.set_properties(**{'background-color': 'black',
                           'color': 'white',
                           'border-color': 'white'}))

 20%|██        | 2/10 [53:20<3:33:13, 1599.17s/it]

In [ ]:
#@title
instructions = ipywidgets.widgets.HTML('Please find attached below the Scores assigned:')
display(instructions)
df_test.to_csv("Scores.csv", index = False)
display(create_download_link(df_test, "Scores", "Scores.csv"))

In [ ]:
#@title
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'''))
instructions = ipywidgets.widgets.HTML('Check below Pivot Table run for Scores')
display(instructions)
pivot_ui(df_test, outfile_path='pivottablejs.html')
display(HTML('pivottablejs.html'))
for i in range(20):
  print("")

<IPython.core.display.Javascript object>

HTML(value='Check below Pivot Table run for Scores')